###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo k2wp-tdv7.csv

Contiene información sobre incautaciones de insumos líquidos. Incluye el número de galones de sustancias químicas empleadas para la fabricación de drogas ilícitas, incautados por la fuerza pública.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/k2wp-tdv7.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad,unidad
0,2024-07-31T00:00:00.000,52.0,NARIÑO,52356,IPIALES,160.0,GALÓN
1,2024-07-31T00:00:00.000,20.0,CESAR,20238,EL COPEY,10571.0,GALÓN
2,2024-07-31T00:00:00.000,20.0,CESAR,20238,EL COPEY,11702.0,GALÓN
3,2024-07-31T00:00:00.000,27.0,CHOCÓ,27001,QUIBDO,300.0,GALÓN
4,2024-07-31T00:00:00.000,47.0,MAGDALENA,47288,FUNDACION,11500.0,GALÓN


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99709 entries, 0 to 99708
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   fecha_hecho   99709 non-null  object 
 1   cod_depto     99706 non-null  float64
 2   departamento  99700 non-null  object 
 3   cod_muni      99706 non-null  object 
 4   municipio     99700 non-null  object 
 5   cantidad      99709 non-null  float64
 6   unidad        99709 non-null  object 
dtypes: float64(2), object(5)
memory usage: 5.3+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [6]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad']
liquid_inputs_confiscations_MinDefensa = df[relevant_cols]
liquid_inputs_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99709 entries, 0 to 99708
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cod_muni      99706 non-null  object 
 1   departamento  99700 non-null  object 
 2   municipio     99700 non-null  object 
 3   fecha_hecho   99709 non-null  object 
 4   cantidad      99709 non-null  float64
 5   unidad        99709 non-null  object 
dtypes: float64(1), object(5)
memory usage: 4.6+ MB


### Convertir columna fecha_hecho a tipo date

In [7]:
# Hacer una copia explícita del DataFrame
liquid_inputs_confiscations_MinDefensa_ = liquid_inputs_confiscations_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
liquid_inputs_confiscations_MinDefensa_['fecha_hecho'] = pd.to_datetime(liquid_inputs_confiscations_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
liquid_inputs_confiscations_MinDefensa_['fecha_hecho'] = liquid_inputs_confiscations_MinDefensa_['fecha_hecho'].dt.year

In [8]:
liquid_inputs_confiscations_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [9]:
liquid_inputs_confiscations_MinDefensa_.isnull().sum()

,0
cod_muni,3
departamento,9
municipio,9
fecha_hecho,0
cantidad,0
unidad,0


Nota: Los nulos los abordamos más adelante

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [10]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(liquid_inputs_confiscations_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['NARIÑO' 'CESAR' 'CHOCÓ' 'MAGDALENA' 'META' 'NORTE DE SANTANDER'
 'PUTUMAYO' 'SUCRE' 'VALLE DEL CAUCA' 'CAQUETÁ' 'CUNDINAMARCA' 'GUAVIARE'
 'SANTANDER' 'TOLIMA' 'BOYACÁ' 'CÓRDOBA' 'BOLÍVAR' 'CAUCA' 'LA GUAJIRA'
 'CASANARE' 'GUAINÍA' 'ANTIOQUIA' 'VICHADA' 'QUINDÍO' 'ECUADOR'
 'RISARALDA' nan 'AMAZONAS' 'CALDAS' 'HUILA' 'ATLÁNTICO' 'ARAUCA'
 'BOGOTÁ D.C.' 'CORDOBA' 'BOLIVAR' 'ATLANTICO' 'CHOCO' 'BOYACA' 'CAQUETA'
 'BOGOTA D.C.' 'GUAINIA' 'VAUPES' 'PERU' 'QUINDIO' 'VAUPÉS' 'VENEZUELA'
 'SAN ANDRÉS ISLAS']

Categorías en la columna 'municipio':
['IPIALES' 'EL COPEY' 'QUIBDO' 'FUNDACION' 'VILLAVICENCIO'
 'SAN ANDRES DE TUMACO' 'EL ZULIA' 'HACARI' 'PUERTO ASIS' 'SAN MIGUEL'
 'VALLE DEL GUAMUEZ' 'GUARANDA' 'MAJAGUAL' 'BUENAVENTURA' 'BUGALAGRANDE'
 'YOTOCO' 'CURILLO' 'LA GLORIA' 'ALBAN' 'PUERTO SALGAR' 'EL RETORNO'
 'ALGARROBO' 'GRANADA' 'GUAMAL' 'PUERTO RICO' 'BARBACOAS'
 'BARRANCABERMEJA' 'PUERTO PARRA' 'SABANA DE TORRES' 'VENADILLO' 'ZARZAL'
 'EL PA

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [11]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [12]:
# Aplicar la función a todas las columnas categóricas
liquid_inputs_confiscations_MinDefensa_[categorical_col] = liquid_inputs_confiscations_MinDefensa_[categorical_col].astype(str)
for col in categorical_col:
    liquid_inputs_confiscations_MinDefensa_[col] = liquid_inputs_confiscations_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [13]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df liquid_inputs_confiscations_MinDefensa_

In [15]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
liquid_inputs_confiscations_MinDefensa_['cod_muni'] = liquid_inputs_confiscations_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = liquid_inputs_confiscations_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [16]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 6 3 7]


In [17]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,99680,0.999709
1,7,25,0.000251
2,3,3,0.000030
3,6,1,0.000010


In [18]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = liquid_inputs_confiscations_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    52356
1    20238
2    20238
3    27001
4    47288

Muestra de registros con longitud 7:
     cod_muni
6014  2180101
6447  2180101
6501  2180101
6749  2180101
7634  2180101

Muestra de registros con longitud 3:
     cod_muni
1075      nan
1076      nan
6994      nan

Muestra de registros con longitud 6:
    cod_muni
963   01-593



In [22]:
# Verificar longitudes de 'cod_mupi' igual a 3, 6 y 7 dígitos
cod_muni_3_6_7 = liquid_inputs_confiscations_MinDefensa_[
    (liquid_inputs_confiscations_MinDefensa_['cod_muni'].apply(len) == 3) |
    (liquid_inputs_confiscations_MinDefensa_['cod_muni'].apply(len) == 6) |
    (liquid_inputs_confiscations_MinDefensa_['cod_muni'].apply(len) == 7)
]
cod_muni_3_6_7['departamento'].value_counts()

,count
departamento,
ECUADOR,18
NAN,9
PERU,2


Nota: Como se aprecia los 'cod_mun' con longitudes diferentes a 5 dígitos, que en total suman 29 registros de un total de 99.709, corresponden a incautaciones hechas en el exterior y NAN. Procedemos a excluirlas porque para efectos del proyecto no son importantes

In [23]:
liquid_inputs_confiscations_MinDefensa_ = liquid_inputs_confiscations_MinDefensa_[liquid_inputs_confiscations_MinDefensa_['cod_muni'].apply(len) == 5]

Nota: El 100%  de los codigos que identifican a los municipios en el df liquid_inputs_confiscations_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [24]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [25]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [26]:
# Comparar listas de departamento
compare_lists(liquid_inputs_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en liquid_inputs_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en liquid_inputs_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'VENEZUELA', 'PERU', 'SAN ANDRES ISLAS'}


  Nota: Aún figuran en el df incautaciones hechas en el exterior: 'VENEZUELA', 'PERU', además el departamento SAN ANDRES ISLAS está escrito en forma diferente a su nombre real
  
  - Solucionar discrepancias en nombres de departamento SAN ANDRES ISLAS

In [27]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe liquid_inputs_confiscations_MinDefensa_
liquid_inputs_confiscations_MinDefensa_['departamento'] = liquid_inputs_confiscations_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [28]:
# Comparar listas de departamento
compare_lists(liquid_inputs_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en liquid_inputs_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en liquid_inputs_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'VENEZUELA', 'PERU'}


In [29]:
# Verificar cuantos registros figuran a nombre de 'VENEZUELA', 'PERU', 'BRASIL', 'ESPANA'
confiscation_overseas = liquid_inputs_confiscations_MinDefensa_[liquid_inputs_confiscations_MinDefensa_['departamento'].isin(['VENEZUELA', 'PERU'])]
confiscation_overseas['departamento'].value_counts()

,count
departamento,
VENEZUELA,6
PERU,2


In [30]:
# Se procede a eliminar esos registros
liquid_inputs_confiscations_MinDefensa_ = liquid_inputs_confiscations_MinDefensa_[~(liquid_inputs_confiscations_MinDefensa_['departamento'].isin(['VENEZUELA', 'PERU']))]

In [31]:
# Comparar nuevamente las listas de departamento
compare_lists(liquid_inputs_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en liquid_inputs_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en liquid_inputs_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [32]:
# Comparar listas de códigos
compare_lists(liquid_inputs_confiscations_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en liquid_inputs_confiscations_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en liquid_inputs_confiscations_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [33]:
liquid_inputs_confiscations_MinDefensa_[liquid_inputs_confiscations_MinDefensa_['cod_muni'] == '94663']

,cod_muni,departamento,municipio,fecha_hecho,cantidad,unidad
74153,94663,GUAINIA,MAPIRIPANA,2015,70.6,GALON


In [34]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name
1092,94001,GUAINIA,INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL
1098,94887,GUAINIA,PANA PANA
1099,94888,GUAINIA,MORICHAL


In [35]:
# Como no hay ningun en Guania de nombre MAPIRIAPANA eliminamos el registro
liquid_inputs_confiscations_MinDefensa_ = liquid_inputs_confiscations_MinDefensa_[~(liquid_inputs_confiscations_MinDefensa_['cod_muni']== '94663')]

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [36]:
# Comparar listas de códigos
compare_lists(liquid_inputs_confiscations_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en liquid_inputs_confiscations_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en liquid_inputs_confiscations_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [37]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_liquid_inputs_confiscations_MinDefensa = liquid_inputs_confiscations_MinDefensa_.drop(columns=columns_to_drop)

In [38]:
# Adicionar columna para trazabilidad de la fuente
final_liquid_inputs_confiscations_MinDefensa['source_id'] = 32

In [39]:
final_liquid_inputs_confiscations_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'unidad', 'source_id'], dtype='object')

In [40]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_liquid_inputs_confiscations_MinDefensa.rename(columns=translation_map, inplace=True)

In [41]:
#Estructura final del dataset a integrar a la base de datos
final_liquid_inputs_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99671 entries, 0 to 99708
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   dane_code         99671 non-null  object 
 1   year_of_incident  99671 non-null  int32  
 2   amount            99671 non-null  float64
 3   unit              99671 non-null  object 
 4   source_id         99671 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 4.2+ MB


## Salvar en archivo csv en el drive

In [42]:
# Guardar en archivos CSV en el drive
final_liquid_inputs_confiscations_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/liquid_inputs_confiscations_MinDefensa.csv', index=False)